In [1]:
import pandas as pd
import os
import gzip
from pathlib import Path
import numpy as np

In [8]:
ipc_to_sdg_mapping = {
    "A01B": [2],
    "A01C": [2],
    "A61K": [3],
    "A61P": [3],
    "B01D": [6, 12],
    "B60L": [11, 13],
    "C02F": [6],
    "C07D": [3],
    "C12N": [3, 9, 15],
    "E03B": [6],
    "F03D": [7, 13],
    "G06F": [9],
    "G01N": [3, 9],
    "H01L": [7],
    "Y02A": [6, 13],
    "Y02E": [7, 13],
    "Y02T": [11, 13],
    "Y02W": [11, 12, 13],
    "B82Y": [3, 9],
    "C10G": [7, 12],
    "A23L": [2, 3],
    "F24J": [7],
    "C01B": [6, 12],
    "E21B": [7, 12],
    "F01K": [7],
}

In [9]:
def process_files():
    # Set up directories
    input_dir = Path("/home/jovyan/epo-sdg/epo_chunks")
    output_dir = Path("/home/jovyan/epo-sdg/output")
    
    # Create output directory if it doesn't exist
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Get all .csv.gz files
    gz_files = list(input_dir.glob("*.csv.gz"))
    print(f"Found {len(gz_files)} .csv.gz files to process")
    
    # Define output file
    output_file = output_dir / "sdg_mapped_patents.csv"
    
    # Process each file and append results incrementally
    total_matched_rows = 0
    first_file = True
    
    for i, gz_file in enumerate(gz_files):
        print(f"Processing file {i+1}/{len(gz_files)}: {gz_file.name}")
        
        try:
            # Read compressed CSV file
            with gzip.open(gz_file, 'rt') as f:
                df = pd.read_csv(f)
            
            # Process the IPC column and find matches
            matched_rows = process_ipc_codes(df)
            
            if len(matched_rows) > 0:
                # Write to output file (with headers only for first file)
                matched_rows.to_csv(
                    output_file, 
                    mode='w' if first_file else 'a',
                    header=first_file,
                    index=False
                )
                
                # Update counters
                total_matched_rows += len(matched_rows)
                print(f"  Found {len(matched_rows)} matching rows, total: {total_matched_rows}")
                
                # Update first_file flag
                if first_file:
                    first_file = False
            else:
                print(f"  No matching rows found")
                
        except Exception as e:
            print(f"  Error processing {gz_file.name}: {str(e)}")
            continue
    
    # Print final statistics
    if total_matched_rows > 0:
        print(f"\nProcessing complete. Saved {total_matched_rows} rows to {output_file}")
        
        # Read back the complete file for summary statistics
        try:
            final_df = pd.read_csv(output_file)
            print_summary(final_df)
        except Exception as e:
            print(f"Error generating summary: {str(e)}")
    else:
        print("\nNo matching patents found across all files")

In [10]:
def process_ipc_codes(df):
    """Process a dataframe and find rows with matching IPC codes"""
    matched_rows = []
    
    # Check if 'ipc' column exists
    if 'ipc' not in df.columns:
        print("  Warning: 'ipc' column not found in dataframe")
        return pd.DataFrame()
    
    for idx, row in df.iterrows():
        ipc_codes = str(row['ipc']).split(',') if pd.notna(row['ipc']) else []
        
        # Clean up IPC codes (remove whitespace)
        ipc_codes = [code.strip() for code in ipc_codes]
        
        # Check for matches with our mapping
        matched_sdgs = set()
        matched_ipc_codes = []
        matched_details = []  # To store the full details of matched codes
        
        for full_ipc_code in ipc_codes:
            # Extract the main IPC code (first 4 characters) for matching
            # Most IPC codes have format like A61K31/4745 where A61K is the main category
            main_ipc_code = None
            
            if len(full_ipc_code) >= 4:
                # Extract first 4 characters (class + subclass)
                main_ipc_code = full_ipc_code[:4]
            
            if main_ipc_code in ipc_to_sdg_mapping:
                matched_sdgs.update(ipc_to_sdg_mapping[main_ipc_code])
                matched_ipc_codes.append(main_ipc_code)
                matched_details.append(full_ipc_code)  # Store the full IPC code for reference
        
        # If we found any matches, create a new row
        if matched_sdgs:
            new_row = row.copy()
            
            # Add SDG binary columns
            for sdg_num in range(18):  # 0 to 17
                if sdg_num == 0:
                    # SDG_0 is 1 if no SDGs are assigned (shouldn't happen in our case)
                    new_row[f'sdg_{sdg_num}'] = 1 if len(matched_sdgs) == 0 else 0
                else:
                    # SDG_1 to SDG_17
                    new_row[f'sdg_{sdg_num}'] = 1 if sdg_num in matched_sdgs else 0
            
            # Add additional useful columns
            new_row['matched_ipc_codes'] = ','.join(matched_ipc_codes)
            new_row['matched_full_codes'] = ','.join(matched_details)
            new_row['assigned_sdgs'] = ','.join(map(str, sorted(matched_sdgs)))
            
            matched_rows.append(new_row)
    
    return pd.DataFrame(matched_rows) if matched_rows else pd.DataFrame()

In [11]:
def print_summary(df):
    """Print a summary of the processed data"""
    print("\n=== SUMMARY ===")
    print(f"Total patents processed: {len(df)}")
    
    # Count patents per SDG
    print("\nPatents per SDG:")
    for sdg_num in range(1, 18):
        count = df[f'sdg_{sdg_num}'].sum()
        print(f"  SDG {sdg_num}: {count} patents")
    
    # Find patents with multiple SDGs
    sdg_columns = [f'sdg_{i}' for i in range(1, 18)]
    df['total_sdgs'] = df[sdg_columns].sum(axis=1)
    multi_sdg_patents = (df['total_sdgs'] > 1).sum()
    print(f"\nPatents assigned to multiple SDGs: {multi_sdg_patents}")
    
    # Most common IPC main categories
    print("\nMost common matched IPC main categories:")
    all_matched_codes = []
    for codes in df['matched_ipc_codes']:
        all_matched_codes.extend(codes.split(','))
    
    from collections import Counter
    code_counts = Counter(all_matched_codes)
    for code, count in code_counts.most_common(10):
        print(f"  {code}: {count} occurrences")
        
    # Sample of detailed IPC codes
    print("\nSample of detailed IPC codes that were matched:")
    sample_details = []
    for details in df['matched_full_codes'][:100]:  # Look at first 100 rows for sampling
        sample_details.extend(details.split(','))
    
    sample_details = list(set(sample_details))  # Remove duplicates
    if len(sample_details) > 15:
        sample_details = sample_details[:15]  # Show at most 15 examples
        
    for code in sample_details:
        print(f"  {code}")

In [ ]:
if __name__ == "__main__":
    process_files()

Found 49 .csv.gz files to process
Processing file 1/49: epo_0033.csv.gz
  Found 3258 matching rows, total: 3258
Processing file 2/49: epo_0049.csv.gz
  Found 2050 matching rows, total: 5308
Processing file 3/49: epo_0007.csv.gz
  Found 4975 matching rows, total: 10283
Processing file 4/49: epo_0022.csv.gz
  Found 4250 matching rows, total: 14533
Processing file 5/49: epo_0003.csv.gz
  Found 5413 matching rows, total: 19946
Processing file 6/49: epo_0023.csv.gz
  Found 4204 matching rows, total: 24150
Processing file 7/49: epo_0017.csv.gz
  Found 5871 matching rows, total: 30021
Processing file 8/49: epo_0029.csv.gz
  Found 3812 matching rows, total: 33833
Processing file 9/49: epo_0046.csv.gz
  Found 7317 matching rows, total: 41150
Processing file 10/49: epo_0016.csv.gz
  Found 3112 matching rows, total: 44262
Processing file 11/49: epo_0047.csv.gz
  Found 4984 matching rows, total: 49246
Processing file 12/49: epo_0019.csv.gz
  Found 2829 matching rows, total: 52075
Processing file 1

In [2]:
import pandas as pd

# Read the big CSV
df = pd.read_csv('/home/jovyan/epo-sdg/output/sdg_mapped_patents.csv')

# Take a random sample of, say, 500 rows (without replacement)
sampled = df.sample(n=250, random_state=42)

# Write out the sample
sampled.to_csv('/home/jovyan/epo-sdg/output/sample_dataset.csv', index=False)


/tmp/ipykernel_266/663973732.py:4: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/jovyan/epo-sdg/output/sdg_mapped_patents.csv')


In [ ]:
import csv
import sys
import gzip
from pathlib import Path

# ↑ Add this right after importing csv:
csv.field_size_limit(sys.maxsize)  # bump the max field size to the platform limit

def split_and_compress_csv(input_csv: Path, chunk_size: int = 20_000) -> bool:
    """
    Splits the input CSV into chunks of `chunk_size` rows,
    compresses each chunk to .csv.gz (keeping the header),
    and deletes the original CSV when done.
    """
    if not input_csv.exists():
        print(f"Error: Input file {input_csv} not found.")
        return False

    try:
        with input_csv.open('r', newline='') as f_in:
            reader = csv.reader(f_in)
            header = next(reader)
            file_index = 1
            buffer = []

            for row in reader:
                buffer.append(row)
                if len(buffer) >= chunk_size:
                    _write_chunk(buffer, header, input_csv.parent, input_csv.stem, file_index)
                    file_index += 1
                    buffer.clear()

            # write any remaining rows
            if buffer:
                _write_chunk(buffer, header, input_csv.parent, input_csv.stem, file_index)

        # delete the original large CSV
        print(f"Deleting original file {input_csv}...")
        input_csv.unlink()
        print("All done.")
        return True

    except Exception as e:
        print(f"Error during splitting/compression: {e}")
        return False


def _write_chunk(rows: list, header: list, out_dir: Path, stem: str, index: int):
    """
    Write one chunk (including header) to a gzip-compressed CSV,
    naming it like `{stem}_0001.csv.gz`.
    """
    chunk_name = f"{stem}_{index:04d}.csv.gz"
    chunk_path = out_dir / chunk_name
    print(f"  → Writing chunk #{index} to {chunk_path} ({len(rows)} rows)…")

    with gzip.open(chunk_path, 'wt', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(header)
        writer.writerows(rows)

    size_mb = chunk_path.stat().st_size / (1024 * 1024)
    print(f"    • Compressed size: {size_mb:.2f} MB")


if __name__ == "__main__":
    input_path = Path("/home/jovyan/epo-sdg/output/sdg_mapped_patents.csv")
    split_and_compress_csv(input_path, chunk_size=20_000)


  → Writing chunk #1 to /home/jovyan/epo-sdg/output/sdg_mapped_patents_0001.csv.gz (20000 rows)…
